# Import Modules

In [3]:
import numpy as np
import time

# Levenshtein Distance Algorithm

In [16]:
def levenshtein_distance(x: str, y: str) -> int:
    # Matrix to store the corresponding edit distances of substring x with respect to y
    m, n = len(x), len(y)
    D = np.zeros((m + 1, n + 1), dtype=int)

    # Initialize first column with each cell being a number corresponding to the position of each character of x
    for i in range(m + 1):
        D[i, 0] = i

    # Initialize first row with each cell being a number corresponding to the position of each character of y
    for j in range(n + 1):
        D[0, j] = j

    # Iterate through each substring in x and compare it with each substring of y
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            # If the characters are different (AKA) then the cost for substitution is 1. Else its 0
            cost = 1
            if x[i - 1] == y[j - 1]:
                cost = 0

            # For the minimum edit distance between substring of x up to i and substring of y up to j,
            # find the minimum between:
            # 1. Cost to delete: D[i - 1, j] + 1
            # 2. Cost to insert: D[i, j - 1] + 1
            # 3. Cost to substitute: D[i - 1, j - 1] + cost
            D[i, j] = min(D[i - 1, j] + 1, D[i, j - 1] + 1, D[i - 1, j - 1] + cost)

    # The minimum of edit distance for x and y is the element in the last column and last row of D
    return D[m, n]

# Levenshtein Distance with Array instead of Matrix

In [18]:
def arr_levenshtein_distance(x: str, y: str) -> int:
    # We choose the string with the shorter length to memoize the calculations
    m, n = len(x), len(y)

    if m > n:
        x, y = y, x
        m, n = n, m

    # Initialize arrays for previous and current rows
    prev = np.arange(m + 1, dtype=int)
    current = np.zeros(m + 1, dtype=int)

    # Iterate through each substring of the longer string up to index i and compare it to each substring of the shorter string
    for i in range(1, n + 1):
        current[0] = i

        # For the minimum edit distance between substring of the longer string up to i and substring of the shorter substring up to j,
        # find the minimum between:
        # 1. Cost to delete: prev[j] + 1
        # 2. Cost to insert: current[j - 1] + 1
        # 3. Cost to substitute: prev[j - 1] + (0 if the characters are the same else 1)
        for j in range(1, m + 1):
            deletion_cost = prev[j] + 1
            insertion_cost = current[j - 1] + 1
            substitution_cost = prev[j - 1] + (0 if x[j - 1] == y[i - 1] else 1)

            current[j] = min(insertion_cost, deletion_cost, substitution_cost)

        # The current array will be the previous array of the next iteration. In truth, the current array doesn't need to be changed
        # as it will be overwritten in the next iteration anyway
        prev = current.copy()

    # The minimum edit distance can be found in prev[m]. This is the equivalent of D[m, n]
    return prev[m]

# Benchmarking